# 📊 Validação de Modelos de Séries Temporais - Backtesting (Walk-Forward)

## 🎯 Objetivo Executivo
Este notebook é responsável pela **validação robusta** dos modelos de previsão. Utiliza a metodologia **Walk-Forward** (janela deslizante) para simular o desempenho do modelo no passado.

## 🛠️ Metodologia
1.  **Strict Mode**: Garante zero vazamento de dados.
2.  **Múltiplos Modelos**: Avalia LightGBM, TFT, Prophet, etc.
3.  **Métricas**: Calcula RMSE e MAPE.

---

# 📊 Validação de Modelos de Séries Temporais - CVC Lojas

## 🎯 Objetivo Executivo
Este notebook tem como objetivo realizar a **validação robusta (Backtesting)** de múltiplos algoritmos de previsão de vendas para as lojas da CVC. O processo simula cenários reais do passado para garantir que o modelo escolhido tenha performance consistente ao longo do tempo, e não apenas em um único período de teste.

## 🛠️ Metodologia: Walk-Forward Validation (Strict Mode)
Diferente da divisão tradicional (Treino/Teste), utilizamos a estratégia de **Walk-Forward** (Janela Deslizante):
1.  O modelo treina com dados até uma data de corte (ex: Dez/2024).
2.  Faz a previsão para o mês seguinte (ex: Jan/2025).
3.  A janela avança 1 mês, o modelo retreina com os dados reais de Jan/2025 e prevê Fev/2025.
4.  Isso se repete por 12 meses (Folds), gerando métricas de erro (RMSE, SMAPE) para cada mês.

> **Nota:** O modo "Strict" garante que **nenhum dado do futuro** (vazamento de dados) seja acessível ao modelo durante o treino, simulando fielmente a produção.

---

## 🤖 Estratégia de Modelos (Model)
A pipeline avalia automaticamente duas classes de algoritmos via biblioteca **Darts**:

### 1. Machine Learning Clássico (Regressores)
* **Linear Regression:** Baseline simples para capturar tendências lineares.
* **Random Forest:** Captura não-linearidades e interações complexas.
* **LightGBM / XGBoost / CatBoost:** Modelos baseados em *Gradient Boosting*, estado da arte para dados tabulares e séries temporais com covariáveis.

### 2. Deep Learning (SOTA - State of the Art)
* **TFT (Temporal Fusion Transformer):** Modelo de atenção que aprende a importância de cada variável ao longo do tempo.
* **N-BEATS:** Rede neural baseada em blocos de tendência e sazonalidade.
* **Transformer:** Arquitetura clássica de *Attention* adaptada para séries temporais.
* **BlockRNN (LSTM):** Redes recorrentes para capturar dependências de longo prazo.
* **TCN (Temporal Convolutional Network):** Convoluções causais para capturar padrões locais e globais.

---

## 🏛️ Arquitetura e Governança (Databricks Unity Catalog)
Este notebook implementa uma arquitetura híbrida para conformidade com o Unity Catalog:

| Componente | Local de Armazenamento | Função |
| :--- | :--- | :--- |
| **Experimentos** | `Workspace/Users/...` | Armazena métricas, gráficos e logs de execução (evita erro de path do UC). |
| **Registro de Modelos** | **Unity Catalog** (`ds_dev.cvc_val`) | O modelo final (`.pkl`) é versionado e governado oficialmente no catálogo. |
| **Assinatura (Signature)** | **Enforced** | Todos os modelos possuem contrato de entrada/saída (`long` -> `double`) validado para evitar erros de tipagem no serving. |

## 📥 Dados de Entrada
* **Target:** `bip_vhistorico_targuet_loja` (Vendas históricas).
* **Covariáveis Futuras:** `bip_vhistorico_feriados_loja` (Calendário nacional/regional).
* **Covariáveis Globais:** `bip_vhistorico_suporte_canal_loja` (Indicadores macroeconômicos e campanhas).

In [0]:
# Configuração de otimização do Spark (Delta Lake)
# --- CONFIGURAÇÕES GLOBAIS DE OTIMIZAÇÃO (BEST PRACTICES) ---
# Ativa otimização automática de gravações e compactação
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")

In [0]:
# Importação de bibliotecas essenciais
# --- IMPORTS (REFATORED) ---
%load_ext autoreload
%autoreload 2

import sys
import pickle
import os
sys.path.append(os.getcwd())

from src.validation.config import Config
from src.validation.data import DataIngestion 
from src.validation.pipeline import ProjectPipeline
from src.validation.trainer import ModelTrainer
from darts import TimeSeries
from darts.dataprocessing.pipeline import Pipeline
from darts.dataprocessing.transformers import (
    Scaler,
    StaticCovariatesTransformer,
    MissingValuesFiller
)
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.models import (
    TFTModel,
    NBEATSModel,
    TransformerModel,
    LinearRegressionModel,
    LightGBMModel,
    XGBModel,
    CatBoostModel,
    RandomForest,
    BlockRNNModel,
    RNNModel,
    TCNModel
)
from darts.metrics import mape, mse, rmse, r2_score, smape
from pytorch_lightning.callbacks import EarlyStopping

# Bibliotecas Padrão
import pandas as pd
import numpy as np
import mlflow

# Ingestão Imports
from databricks.feature_engineering import FeatureEngineeringClient, FeatureLookup
import pyspark.sql.functions as F
from darts import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries

In [0]:
# Inicialização da configuração centralizada (parâmetros do widget)
if spark is None:
    raise RuntimeError("Spark Session not available.")

config = Config()
config.spark_session = spark

In [0]:
print(f"⏱️ PERÍODO DE TREINO: {config.DATA_START} até {config.TRAIN_END_DATE}")
print(f"📥 PERÍODO DE INGESTÃO (Covariáveis): Até {config.INGESTION_END}")
print(f"🔍 JANELA DE TESTE (BACKTEST): {config.VAL_START_DATE} até {config.TRAIN_END_DATE}")

In [0]:
# Instanciação da classe de Ingestão: Prepara os dados brutos e Feature Store
# --- EXECUÇÃO DO PIPELINE (OTIMIZADO COM FEATURE STORE) ---
print(f"🚀 Iniciando Pipeline v{config.VERSION} (Walk-Forward Strict Mode)")
ingestion = DataIngestion(spark, config)

# No bloco de execução:
# 1. Busca Unificada (Feature Store + Spark ETL)
df_spark_wide = ingestion.create_training_set() # Retorna Spark DF
df_support_global = ingestion.get_global_support() # Retorna Pandas (pois é pequeno)

# 2. Construção dos Objetos Darts (Aqui ocorre o toPandas)
raw_series, raw_covs = ingestion.build_darts_objects(df_spark_wide, df_support_global)

# --- Daqui para baixo, o código original de treino se mantém igual ---
# 3. SPLIT DE TREINO
train_cutoff_date = pd.Timestamp(config.TRAIN_END_DATE) - pd.Timedelta(days=1)
print(f"✂️ Data corte para treino estático: {train_cutoff_date.date()}")

In [0]:
# Pipeline de pré-processamento: Normaliza os dados (0 a 1)
print("🛠️ Ajustando Pipeline (Scalers)...")
project_pipeline = ProjectPipeline()

# Define a data de corte (train_cutoff_date já deve estar definida como 2024-12-31)
# Filtramos apenas séries que possuem dados ANTES da data de corte
train_for_fit = [
    s.drop_after(train_cutoff_date) 
    for s in raw_series 
    if s.start_time() <= train_cutoff_date
]

# Sincronizamos as covariáveis para as mesmas lojas válidas
cov_for_fit = [
    c.drop_after(train_cutoff_date) 
    for s, c in zip(raw_series, raw_covs) 
    if s.start_time() <= train_cutoff_date
]
if not train_for_fit:
    raise ValueError("Nenhuma loja possui histórico anterior a " + str(train_cutoff_date))

print(f"✅ Ajustando Scalers com {len(train_for_fit)} lojas que possuem histórico.")
project_pipeline.fit(train_for_fit, cov_for_fit)

In [0]:
# Pipeline de pré-processamento: Normaliza os dados (0 a 1)
# --- AJUSTE DO PIPELINE (SCALERS) ---
print("🛠️ Ajustando Pipeline (Scalers) em TODAS as séries...")
project_pipeline = ProjectPipeline()

# Com a Reindexação Universal, TODAS as séries possuem o ponto 'train_cutoff_date'
train_for_fit = [s.drop_after(train_cutoff_date) for s in raw_series]
cov_for_fit = [c.drop_after(train_cutoff_date) for c in raw_covs]

# O fit agora aprende a escala global (considerando zeros das lojas novas)
project_pipeline.fit(train_for_fit, cov_for_fit)

print("🔄 Transformando séries (Scaling)...")
series_scaled_full, cov_scaled_full = project_pipeline.transform(raw_series, raw_covs)

# Criando as fatias de treino estático para os modelos
train_series_static = [s.drop_after(train_cutoff_date) for s in series_scaled_full]
train_cov_static = [c.drop_after(train_cutoff_date) for c in cov_scaled_full]

print("🔄 Preparando targets originais para inversão de escala na validação...")
# Agora usamos a lista completa diretamente
val_series_original = project_pipeline.inverse_transform(series_scaled_full, partial=True)

# --- 4. CONFIGURAÇÃO DE MODELOS ---
lag = config.LAGS
lag_covariantes = config.LAGS_FUTURE
forecast = config.FORECAST_HORIZON
lag_2 = lag + config.FORECAST_HORIZON
dynamic_kernel = 3
EARLY_STOPPER = EarlyStopping(monitor="train_loss", patience=5, min_delta=0.001, mode='min')

models_dict = {
    "LinearRegression": LinearRegressionModel(
        lags=lag, lags_future_covariates=lag_covariantes, 
        output_chunk_length=forecast, multi_models=True
    ),
    "RandomForest": RandomForest(
        lags=lag, lags_future_covariates=lag_covariantes, 
        output_chunk_length=forecast, multi_models=False, random_state=42
    ),
    "LightGBM": LightGBMModel(
        lags=lag, lags_future_covariates=lag_covariantes, 
        output_chunk_length=forecast, multi_models=True, random_state=42,
        device="gpu"  # Ativa GPU no LightGBM
    ),
    "XGBoost": XGBModel(
        lags=lag, lags_future_covariates=lag_covariantes, 
        output_chunk_length=forecast, multi_models=True, random_state=42,
        device="cuda" # Ativa GPU no XGBoost
    ),
    "CatBoost": CatBoostModel(
        lags=lag, lags_future_covariates=lag_covariantes, 
        output_chunk_length=forecast, multi_models=True, random_state=42,
        task_type="GPU" # Ativa GPU no CatBoost
    )
}

if config.N_EPOCHS > 0:
    # Configuração para modelos PyTorch (TFT, NBEATS, etc.)
    pl_trainer_kwargs = {
        "accelerator": "gpu", 
        "devices": 1, 
        "callbacks": [EARLY_STOPPER]
    }
    
    models_dict.update({
        "TFT": TFTModel(
            input_chunk_length=lag_2, output_chunk_length=forecast,
            hidden_size=128, lstm_layers=2, num_attention_heads=4,
            dropout=0.2, batch_size=4, n_epochs=config.N_EPOCHS,
            add_relative_index=True, random_state=42, pl_trainer_kwargs=pl_trainer_kwargs
        ),
        "NBEATS": NBEATSModel(
            input_chunk_length=lag_2, output_chunk_length=forecast,
            generic_architecture=True, num_stacks=3, num_blocks=3,
            num_layers=4, layer_widths=256, batch_size=4,
            n_epochs=config.N_EPOCHS, random_state=42, pl_trainer_kwargs=pl_trainer_kwargs
        ),
        "Transformer": TransformerModel(
            input_chunk_length=lag_2, output_chunk_length=forecast,
            d_model=128, nhead=4, num_encoder_layers=3,
            num_decoder_layers=3, dim_feedforward=256, dropout=0.2,
            batch_size=4, n_epochs=config.N_EPOCHS, random_state=42, pl_trainer_kwargs=pl_trainer_kwargs
        ),
        "BlockRNN": BlockRNNModel(
            model='LSTM', input_chunk_length=lag_2, output_chunk_length=forecast,
            hidden_dim=128, n_rnn_layers=2, dropout=0.2,
            batch_size=4, n_epochs=config.N_EPOCHS, random_state=42, pl_trainer_kwargs=pl_trainer_kwargs
        ),
        "TCN": TCNModel(
            input_chunk_length=lag_2, output_chunk_length=forecast,
            kernel_size=dynamic_kernel, num_filters=lag_2,
            num_layers=None, dilation_base=2, dropout=0.2,
            batch_size=4, n_epochs=config.N_EPOCHS, random_state=42, pl_trainer_kwargs=pl_trainer_kwargs
        )
    })

# --- INÍCIO DO TREINO E VALIDAÇÃO ---
trainer = ModelTrainer(config, models_dict)
trainer.train_evaluate_walkforward(
    train_series_static=train_series_static,
    train_covs_static=train_cov_static,
    full_series_scaled=series_scaled_full,    # Lista completa escalonada
    full_covariates_scaled=cov_scaled_full,   # Lista completa escalonada
    val_series_original=val_series_original, # Lista completa original (para erro real)
    target_pipeline=project_pipeline
)

print("✅ Processo Finalizado com Sucesso.")